In [ ]:
import torch
import torch.nn as nn 
from DataLoader import DataLoader 
import torch.optim as optim
from Models import RotationNN
from utils import rotate

In [ ]:
image_size = 512
num_samples = 100

loader = DataLoader(image_size, num_samples)
model = RotationNN(image_size)
criterion = nn.BCELoss() 
optimizer = optim.SGD(model.parameters(), lr=0.01) 

In [ ]:
epochs = 1000
for epoch in range(epochs):
    #todo
    x = loader.__next__()
    y = rotate(y)
    y_pred = model(x)

    loss = criterion(y_pred.squeeze(), y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')